This notebook shows how to design a transmon pocket (single pad design) in qiskit metal. The notebook only need **TransmonPocket_Single** function from a relative path '../../resources'.

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
#This imports the necessary path to draw the single pad transmon design. 
import sys
sys.path.append('Customized_Components')
from rounded_single_pad import Round_TransmonPocket_Single as transmon
import Transmon_property as trans_p
import Transmon_specifications as jj

import shapely
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

In [ ]:
import os
import shutil
import time

In [ ]:
import numpy as np
import astropy.units as u
import astropy.constants as c

import pandas as pd
import matplotlib.pyplot as plt

# %matplotlib inline

In [ ]:
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings

from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
# from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
# from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond

from qiskit_metal.qlibrary.tlines.mixed_path import RouteMixed
# from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight

from qiskit_metal.analyses.quantization import LOManalysis
from qiskit_metal.analyses.quantization import EPRanalysis

# from qiskit_metal.toolbox_metal.parsing import *

In [ ]:
# from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
# from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround


In [ ]:
# pd.read_csv("Frequencies from metal pad size.csv",).plot(x= 'Pad_len (mm)', y = 'EPR Frequency (GHz)')
# plt.ylabel('Frequency (GHz)')
# plt.show()

The target frequency is around 4.2GHz. To get this frequency, we need the pad size to be around 0.275mm.

## Global variables

In [ ]:
phi0 = c.h/2/c.e.si
T = 30*u.mK
epsilon_r = 11.45

In [ ]:
design = designs.DesignPlanar({}, True)
design.chips.main.size['size_x'] = '20 mm'
design.chips.main.size['size_y'] = '10 mm'

design.variables['cpw_width'] = '15 um'
design.variables['cpw_gap'] = '9 um'

design.variables['trace_width'] = '10 um'
design.variables['trace_gap'] = '7 um'

design.variables['pad_buffer_radius'] = '30 um'
design.variables['buffer_resolution'] = '10'
design.variables['connection_pad_buffer_radius'] = '2 um'
# design.chips['main']['material'] = 'Silicon'

gui = MetalGUI(design)

Component-specific variables

In [ ]:


qb_options = dict(
pad_pocket_distance_top = '40um',
connection_pads = dict(
    a = dict(loc_W  = 0, 
            loc_H =  1, 
            pad_gap = '15um',
            pad_height = '15um',
            pad_width = '80um',
            pad_cpw_extent =  '10um',
            pocket_rise = '0um',
            cpw_extend = '0um',
            pocket_extent = '0um',)))

TQ_options = dict(prime_width = design.variables['cpw_width'],
               prime_gap = design.variables['cpw_gap'],
               second_width = design.variables['trace_width'],
               second_gap = design.variables['trace_gap'],
               down_length = '60um',
               coupling_space = '5um',
               open_termination=False,
               hfss_wire_bonds = False,
               q3d_wire_bonds = False)

CPW_options = Dict(trace_width = design.variables['trace_width'],
               trace_gap  = design.variables['trace_gap'],
        total_length='5 mm',
        hfss_wire_bonds = True,
        q3d_wire_bonds = True,
        fillet='30 um',
        lead = dict(start_straight='20um', end_straight = '50um'),
        pin_inputs=Dict(
            start_pin=Dict(component='Q1', pin='a'),
            end_pin=Dict(component='TQ1', pin='second_end')), )

pin_inputs = Dict(
            start_pin=Dict(component='Q1', pin='a'),
            end_pin=Dict(component='TQ1', pin='second_end'))

trans_options = Dict(hfss_wire_bonds = True,
                     q3d_wirebonds = True,
               pin_inputs=Dict(
                 start_pin=Dict(
                     component='TQ1',
                     pin='prime_start'),
                 end_pin=Dict(
                     component='TQ2',
                     pin='prime_end')))


## Design

Single pad transmon

In [ ]:
design.delete_all_components()

In [ ]:
design.overwrite_enabled = True

size = 0.25*u.mm
gap = 30*u.um
size = size.to(u.um)
pocket_width = size+2*gap


op = Dict(
    pos_x = '0',
    pad_height = '{}um'.format(size.value),
    pos_y = '0',
    pad_width = '{}'.format(size),
    pocket_width = '{}'.format(pocket_width), 
    **qb_options)

q1 = transmon(design,'Q1',options = op)

gui.rebuild()
gui.zoom_on_components(['Q1'])


In [ ]:
op = Dict(
    pos_x = '1mm',
    pad_height = '{}um'.format(size.value),
    pos_y = '0',
    pad_width = '{}'.format(size),
    pocket_width = '{}'.format(pocket_width), 
    **qb_options)

q2 = transmon(design,'Q2',options = op)

gui.rebuild()
gui.zoom_on_components(['Q2'])

Coupled Line Tee

In [ ]:
TQ_options['down_length'] = '40 um'
TQ1 = CoupledLineTee(design, 'TQ1', options=dict(pos_x='-0.1 mm',
                                             pos_y='0.5 mm',
                                             mirror = True,
                                             coupling_length='120 um', **TQ_options))
TQ1.options['open_termination'] = False

gui.rebuild()
gui.autoscale()

In [ ]:
TQ2 = CoupledLineTee(design, 'TQ2', options=dict(pos_x='0.9 mm',
                                             pos_y='0.5 mm',
                                             mirror = True,
                                             coupling_length='120 um', **TQ_options))
TQ2.options['open_termination'] = False

gui.rebuild()
gui.autoscale()

In [ ]:
q1.name[-1:]

Function that constructs CPW

In [ ]:
def Construct_CPW(q,TQ,pad_size, offset, extend, Lj, Cj, TQx,TQy, small, TQ_mir, buffer = 150*u.um):
    gap1 = 0.056
    gap = 30*u.um
    size = pad_size.to(u.um)
    pocket_width = size+2*gap
    design.delete_component('cpw_1')
    coupling_len = extend
    q.options['pad_height'] = '{}'.format(size)
    q.options['pad_width'] = '{}'.format(size)
    q.options['pocket_width'] = '{}'.format(pocket_width)
    q.options['connection_pads']['a']['pad_width'] = '{}'.format(coupling_len)
    q.options.hfss_inductance = Lj
    q.options.q3d_inductance =  Lj
    q.options.hfss_capacitance = Cj
    q.options.q3d_capacitance =  Cj



    eig_all.sim.renderer.options['Lj'] = Lj
    eig_all.sim.renderer.options['Cj'] = Cj

    eig_all.sim.setup.vars = Dict(Lj1= Lj, Cj1= Cj)    
    gui.rebuild()
    

    anchors = trans_p.anchor_CPW_round(q,buffer, gap1, 2, small = small, last_offset = offset)
    design.delete_component('cpw_1')
    gui.rebuild()
    TQ.options.pos_x = TQx
    TQ.options.pos_y = TQy
    TQ.options.mirror = TQ_mir
    gui.rebuild()
    
    pin_inputs = Dict(
                start_pin=Dict(component=q.name, pin='a'),
                end_pin=Dict(component=TQ.name, pin='second_end'))

    CPW_options['pin_inputs'] = pin_inputs

    qa = RouteMixed(design, 'cpw_'+q.name[-1:], options = Dict(anchors = anchors, **CPW_options))

    gui.rebuild()

    return qa

    
    # length = trans_p.find_total_len(qa, q1, TQ1, count_extend=False)
    # return (trans_p.find_actual_frequency(length, cpw_pin_width, cpw_gap), length)

Import the file that contains optimal qubit-CPW specs

In [ ]:
data = pd.read_csv('data/educated_guess_0402.csv')

In [ ]:
data

In [ ]:
index = data.keys()

In [ ]:
index

In [ ]:
freqs = data['Target_qb_Freq(GHz)'].values
sizes = list(data[index[2]].values)
Ljs = list((data[index[3]].values).astype(str))
buffers = list((data[index[4]].values))
offsets = list(data[index[5]].values)
coupling_space = np.array([])
TQx = list((data[index[6]].values).astype(str))
TQy = list((data[index[7]].values).astype(str))
small = list(data[index[8]].values)
TQ_mir = list((data[index[9]].values))
cpw_freq_predict = list(data[index[10]].values)

# Simulation of Couplings

Import Qiskit Packages

In [ ]:
from qiskit_metal.analyses.quantization import LOManalysis
from qiskit_metal.analyses.quantization import EPRanalysis

Set the right path for Ansys Screenshots

In [ ]:
day = '0403'
time = '1130'
original = r'C:\Users\slab\Desktop\Wendy-qiskit-code\ansys.png'
path = r'C:\Users\slab\Desktop\Wendy-qiskit-code\Ansys_screenshoots\Simulation'+day+trys

In [ ]:
components  = ['Q1','cpw_1','TQ1','Q2','TQ2','cpw_2','cpw_mid']
nmode = 4
datas = pd.DataFrame()

# Two Qubit design

In [ ]:
design.delete_all_components()

## Draw

### Transmission Line

In [ ]:

# TQ_options['down_length'] = '40 um'

# TQ1 = CoupledLineTee(design, 'TQ1', options=dict(pos_x='-0.23 mm',
#                                              pos_y='0.5 mm', 
#                                              mirror = True,
#                                              coupling_length='120 um', **TQ_options))

# TQ2 = CoupledLineTee(design, 'TQ2', options=dict(pos_x='0.43 mm',
#                                              pos_y='0.5 mm',
#                                              mirror = True,
                                            #  coupling_length='120 um', **TQ_options))

trans_options['pin_inputs']['start_pin']['component'] = 'TQ1'
trans_options['pin_inputs']['start_pin']['pin'] = 'prime_end'
trans_options['pin_inputs']['end_pin']['component'] = 'TQ2'
trans_options['pin_inputs']['end_pin']['pin'] = 'prime_start'
middle = RouteStraight(design, 'CPW_mid', trans_options)

# wb_left = LaunchpadWirebond(design,'wb_left', options = Dict(pos_x = '-0.6mm',pos_y = '0.5mm'))
# wb_left = LaunchpadWirebond(design,'wb_right', options = Dict(pos_x = '10mm',pos_y = '0.5mm',orientation = '180'))

# trans_options['pin_inputs']['start_pin']['component'] = 'wb_left'
# trans_options['pin_inputs']['start_pin']['pin'] = 'tie'
# trans_options['pin_inputs']['end_pin']['component'] = 'TQ1'
# trans_options['pin_inputs']['end_pin']['pin'] = 'prime_start'

# left = RouteStraight(design,'CPW_left', trans_options)

# trans_options['pin_inputs']['start_pin']['component'] = 'wb_right'
# trans_options['pin_inputs']['start_pin']['pin'] = 'tie'
# trans_options['pin_inputs']['end_pin']['component'] = 'TQ2'
# trans_options['pin_inputs']['end_pin']['pin'] = 'prime_end'

# right = RouteStraight(design,'CPW_right', trans_options)


gui.rebuild()
gui.zoom_on_components(['TQ1','TQ2'])

### Qubit 1

In [ ]:
freqs_size = {}
freqs_size['4GHz'] = size1
freqs_size['4.2GHz'] = size2


In [ ]:
size1 = 250*u.um
size2 = 200*u.um
gap = 30*u.um
size1 = size1.to(u.um)
pocket_width1 = size1+2*gap
pocket_width2 = size2+2*gap


q1 = TransmonPocket_Single(design,'Q1', 
                           options = Dict(pocket_width = '{}'.format(pocket_width1),
                                          pad_width = '{}'.format(size1), 
                           pad_height = '{}'.format(size1),
                           pos_x = '0', pox_y = '0', **qb_options))

anchor = trans_p.anchor_CPW(q1, 150*u.um,0.05,2,last_offset = 0.4)
# last = anchor[len(anchor)-1]
# first = anchor[0]
# anchor[len(anchor)-1] = np.array([first[0]-0.05,last[1]])

design.delete_component('cpw1')
pin_inputs['start_pin']['component'] = 'Q1'
pin_inputs['start_pin']['pin'] = 'a'
pin_inputs['end_pin']['component'] = 'TQ1'
pin_inputs['end_pin']['pin'] = 'second_end'
CPW_options['pin_inputs'] = pin_inputs
op = Dict(anchors = anchor, **CPW_options)


cpw1 = RouteMixed(design, 'cpw_1', op)

gui.rebuild()
gui.autoscale()

gui.zoom_on_components(['Q1', 'cpw_1','TQ1'])

Calculate CPW1 length and analytical frequency

In [ ]:
len1 = trans_p.find_total_len(cpw1, q1,TQ1,count_extend=True)
print(len1)
trans_p.find_actual_frequency(len1, 15*u.um, 9*u.um)

### Qubit 2

In [ ]:
x = 0.4/size1*size2
print(x)

In [ ]:
q2 = TransmonPocket_Single(design,'Q2',
                           options = Dict(pocket_width = '{}'.format(pocket_width2),
                                          pad_width = '{}'.format(size2), 
                           pad_height = '{}'.format(size2),
                           pos_x = '0.65', pox_y = '0', **qb_options))


anchor = trans_p.anchor_CPW(q2, 150*u.um,0.05,2,last_offset = x.value)
# last = anchor[len(anchor)-1]
# first = anchor[0]
# anchor[len(anchor)-1] = np.array([first[0]-0.05,last[1]])

design.delete_component('cpw2')
pin_inputs['start_pin']['component'] = 'Q2'
pin_inputs['end_pin']['component'] = 'TQ2'
CPW_options['pin_inputs'] = pin_inputs
op = Dict(anchors = anchor, **CPW_options)


cpw2 = RouteMixed(design, 'cpw_2', op)

gui.rebuild()
gui.zoom_on_components(['Q2', 'cpw_2','TQ2'])

Calculate CPW2 length and analytical frequency

In [ ]:
len2 = trans_p.find_total_len(cpw2, q2,TQ1,count_extend=True)
print(len2)
trans_p.find_actual_frequency(len2, 10*u.um, 7*u.um)

## Simulate

In [ ]:
qcomps = design.components  # short handle (alias)
qcomps['Q1'].options['hfss_inductance'] = 'Lj1'
qcomps['Q1'].options['hfss_capacitance'] = 'Cj1'
qcomps['Q2'].options['hfss_inductance'] = 'Lj2'
qcomps['Q2'].options['hfss_capacitance'] = 'Cj2'



q1.options.hfss_inductance = '13 nH'
q2.options.hfss_inductance = '13 nH'
gui.rebuild()  # line needed to propagate the updates from the qubit instance into the junction design table
gui.autoscale()

In [ ]:
cs = LOManalysis(design, "q3d")
cs.sim.setup.min_passes  = 10
cs.sim.setup.max_passes = 30
cs.sim.setup.freq_ghz = 4

In [ ]:
eig_qb = EPRanalysis(design, 'hfss')
    
eig_qb.sim.setup.max_passes = 15
eig_qb.sim.setup.max_delta_f = 0.2
eig_qb.sim.setup.n_modes = 2
# eig_qb.sim.setup.vars = Dict(Lj1= '13 nH', Cj1= '0 fF')
# eig_qb.sim.setup.vars = Dict(Lj2= '13 nH', Cj2= '0 fF')

eig_qb.sim.renderer.options['x_buffer_width_mm'] = 0.5
eig_qb.sim.renderer.options['y_buffer_width_mm'] = 0.5

### Q1 CPW1

In [ ]:
TQ1.options['open_termination'] = True
gui.rebuild()

cs.sim.run(components=['Q1','cpw_1','TQ1'])

In [ ]:
cs.sim.capacitance_matrix

In [ ]:
TQ1.options['open_termination'] = False
gui.rebuild()

In [ ]:
eig_qb.sim.run(name="Transmons_CPW", components=['Q1', 'cpw_1','TQ1'])

In [ ]:
fs = eig_qb.get_frequencies()

In [ ]:
fs

In [ ]:
eig_qb.sim.renderer.options['Lj1'] = '13nH'
eig_qb.sim.renderer.options['Cj1'] = '0'

In [ ]:
del eig_qb.setup.junctions['jj']

In [ ]:
qcomps = design.components  # short handle (alias)
qcomps['Q1'].options['hfss_inductance'] = 'Lj1'
qcomps['Q1'].options['hfss_capacitance'] = 'Cj1'
gui.rebuild()  # line needed to propagate the updates from the qubit instance into the junction design table
gui.autoscale()

In [ ]:
eig_qb.sim.setup.vars = Dict(Lj1= '13 nH', Cj1= '0 fF')

eig_qb.setup.junctions.jj1 = Dict(rect='JJ_rect_Lj_Q1_rect_jj', line='JJ_Lj_Q1_rect_jj_',
                Lj_variable='Lj1', Cj_variable='Cj1')
eig_qb.setup.sweep_variable = 'Lj1'
eig_qb.run_epr()

### QB2 CPW2

In [ ]:
TQ2.options['open_termination'] = True
gui.rebuild()

cs.sim.run(components=['Q2','cpw_2','TQ2'])

In [ ]:
c_mat  = cs.sim.capacitance_matrix
c_mat

In [ ]:
TQ2.options['open_termination'] = False
gui.rebuild()
cs.sim.run(components=['Q2','cpw_2','TQ2'])

In [ ]:
c_mat  = cs.sim.capacitance_matrix
c_mat

In [ ]:
cq = -c_mat['a_connector_pad_Q2']['pad_top_Q2']
trans_p.freq_from_cap(cq*u.fF)

In [ ]:
eig_qb.sim.run(name="Transmons_CPW", components=['Q2', 'cpw_2','TQ2'])

In [ ]:
fs = eig_qb.get_frequencies()

In [ ]:
fs

In [ ]:
eig_qb.sim.renderer.options['Lj2'] = '13nH'
eig_qb.sim.renderer.options['Cj2'] = '0'

In [ ]:
del eig_qb.setup.junctions['jj']

In [ ]:
qcomps = design.components  # short handle (alias)
qcomps['Q2'].options['hfss_inductance'] = 'Lj2'
qcomps['Q2'].options['hfss_capacitance'] = 'Cj2'
gui.rebuild()  # line needed to propagate the updates from the qubit instance into the junction design table
gui.autoscale()

In [ ]:
eig_qb.sim.setup.vars = Dict(Lj1= '13 nH', Cj1= '0 fF')

eig_qb.setup.junctions.jj1 = Dict(rect='JJ_rect_Lj_Q2_rect_jj', line='JJ_Lj_Q2_rect_jj_',
                Lj_variable='Lj2', Cj_variable='Cj2')
eig_qb.setup.sweep_variable = 'Lj2'
eig_qb.run_epr()

# Simulation of 2 QB

In [ ]:
components_all = ['Q1','Q2','cpw_1','cpw_2','TQ1','TQ2','CPW_left','CPW_right', 'CPW_mid','wb_left','wb_right']

## Capacitance

In [ ]:
c = LOManalysis(design, "q3d")
c.sim.setup.min_passes  = 10
c.sim.setup.max_passes = 30

In [ ]:
c.sim.run(components=['Q1','Q2','cpw_1','cpw_2','TQ1','TQ2','wb_left','wb_right','CPW_left','CPW_right', 'CPW_mid'])
c.sim.capacitance_matrix 

c.sim._get_results_from_renderer()
c_mat = c.sim.capacitance_matrix

In [ ]:
c_mat 

In [ ]:
C = c_mat.values[2:,2:]

In [ ]:
C

## EPR

In [ ]:
eig_all = EPRanalysis(design, 'hfss')

In [ ]:


components = ['Q1','Q2','cpw_1','cpw_2','TQ1','TQ2','CPW_mid','CPW_left','CPW_right']
#'wb_left','wb_right','CPW_left','CPW_right', 

hfss = eig_all.sim.renderer

eig_all.sim.renderer.options['Lj'] = '13nH'
eig_all.sim.renderer.options['wb_threshold'] = '72um'

eig_all.sim.setup.max_passes = 5
eig_all.sim.setup.max_delta_f = 0.05
eig_all.sim.setup.n_modes = 4
eig_all.sim.setup.vars = Dict(Lj1= '13 nH', Cj1= '0 fF',Lj2 = '13nH', Cj2 = '0fF')
eig_all.sim.renderer.options['x_buffer_width_mm'] = 0.5
eig_all.sim.renderer.options['y_buffer_width_mm'] = 0.5
eig_all.sim.run(name="all_freqs", components=components)#, open_terminations=[('Q1', 'a')])
convergence = pd.read_csv('hfss_eig_f_convergence.csv')

In [ ]:
convergence.dropna()

In [ ]:
del eig_all.setup.junctions['jj']
eig_all.setup.junctions.jj1 = Dict(rect='JJ_rect_Lj_Q1_rect_jj', line='JJ_Lj_Q1_rect_jj_',
                Lj_variable='Lj1', Cj_variable='Cj1')
eig_all.setup.junctions.jj2 = Dict(rect='JJ_rect_Lj_Q2_rect_jj', line='JJ_Lj_Q2_rect_jj_',
                Lj_variable='Lj2', Cj_variable='Cj2')
eig_all.setup.sweep_variable = 'Lj1'
eig_all.run_epr()

chi = eig_all.sim.renderer.epr_quantum_analysis.get_chis().to_numpy()[0][0]
freq_EPR= eig_all.sim.renderer.epr_quantum_analysis.get_frequencies().to_numpy()[0][0]

# A huge loop

In [ ]:
xs = np.linspace(0.65,14.5,10)
components = ['Q1','Q2','cpw_1','cpw_2','TQ1','TQ2'] 
components_all = ['Q1','Q2','cpw_1','cpw_2','TQ1','TQ2',
                  'CPW_left','CPW_right', 'CPW_mid','wb_left','wb_right']
#we are not simulating the feedline here since it contains a mode close to the qubit frequency

In [ ]:
c = LOManalysis(design, "q3d")
c.sim.setup.min_passes  = 2
c.sim.setup.percent_error = 10

In [ ]:
eig_all = EPRanalysis(design, 'hfss')
eig_all.sim.setup.min_passes = 5
eig_all.sim.setup.max_delta_f = 0.2

In [ ]:
hfss = eig_all.sim.renderer
original = r'C:\Users\slab\Desktop\Wendy-qiskit-code\ansys.png'
path = r'C:\Users\slab\Desktop\Wendy-qiskit-code\Ansys_screenshoots\Simulation_0304'

debugging

In [ ]:
# eig_all.clear
# renderer_hfss.clean_active_design()

In [ ]:
renderer_hfss = design.renderers.hfss
renderer_q3d = design.renderers.q3d
gds = design.renderers.gds

In [ ]:
renderer_q3d.clean_active_design()
renderer_hfss.clean_active_design()

In [ ]:
renderer_hfss.clean_active_design()

In [ ]:
datas = pd.DataFrame()
mode = 0
x = xs[0]
data = {}
nmodes = 5

data['Pos1_x(mm)'] = 0
data['Pos2_x(mm)'] = x

design.delete_component('cpw_2')

TQ2.options['pos_x'] = x-0.22
q2.options['pos_x'] = x
anchor = trans_p.anchor_CPW(q2, 150*u.um,0.05,2,last_offset = 0.4)
# cpw2.options['anchors'] = anchor
pin_inputs['start_pin']['component'] = 'Q2'
pin_inputs['end_pin']['component'] = 'TQ2'
CPW_options['pin_inputs'] = pin_inputs
op = Dict(anchors = anchor, **CPW_options)
cpw2 = RouteMixed(design, 'cpw_2', op)

gui.rebuild()

c.sim.setup.min_passes  = 10
c.sim.setup.max_passes = 30
c.sim.run(components=components_all)
c.sim.capacitance_matrix 

c.sim._get_results_from_renderer()
c_mat = c.sim.capacitance_matrix

renderer_q3d.clean_active_design()

c1  = c_mat['a_connector_pad_Q1']['pad_top_Q1']
c2  = c_mat['a_connector_pad_Q1']['pad_top_Q2']
c12  = c_mat['pad_top_Q1']['pad_top_Q2']

data['C_Q1_Q2(fF)'] = c12
data['C_Q1_g(fF)'] = c1
data['C_Q2_g(fF)'] = c2




eig_all.sim.renderer.options['Lj'] = '13nH'
eig_all.sim.renderer.options['Lj1'] = '13nH'
eig_all.sim.renderer.options['Lj2'] = '13nH'
eig_all.sim.renderer.options['wb_threshold'] = '72um'


eig_all.sim.setup.n_modes = nmodes
eig_all.sim.setup.vars = Dict(Lj1= '13 nH', Cj1= '0 fF',Lj2 = '13nH', Cj2 = '0fF')
eig_all.sim.renderer.options['x_buffer_width_mm'] = 0.5
eig_all.sim.renderer.options['y_buffer_width_mm'] = 0.5
eig_all.sim.run(name="all_freqs", components=components_all)#, open_terminations=[('Q1', 'a')])
convergence = pd.read_csv('hfss_eig_f_convergence.csv')
conv = convergence.dropna()

ind = list(conv.keys())[1:]
for i in range(nmodes):
    freq = conv[ind[(i)]].values[-1]
    data['Freq'+str(i+1)+'(GHz)'] = freq

#save screenshots for future refrences
for i in range(nmodes):
    hfss.modeler._modeler.ShowWindow()
    hfss.set_mode(i+1,'Setup')
    hfss.plot_ansys_fields('main')
    hfss.save_screenshot()
    hfss.clear_fields(['main'])
    
    actual = r'C:\Users\slab\Desktop\Wendy-qiskit-code\s{}_m{}.png'.format(mode,i)

    os.rename(original, actual)
    dest = shutil.move(actual, path)

    

In [ ]:
#Run the EPR analysis
del eig_all.setup.junctions['jj']
eig_all.setup.junctions.jj1 = Dict(rect='JJ_rect_Lj_Q1_rect_jj', line='JJ_Lj_Q1_rect_jj_',
                Lj_variable='Lj1', Cj_variable='Cj1')
eig_all.setup.junctions.jj2 = Dict(rect='JJ_rect_Lj_Q2_rect_jj', line='JJ_Lj_Q2_rect_jj_',
                Lj_variable='Lj2', Cj_variable='Cj2')
eig_all.setup.sweep_variable = 'Lj1'
eig_all.run_epr()



In [ ]:
#Save the EPR data
chi = eig_all.sim.renderer.epr_quantum_analysis.get_chis().to_numpy()
freq_EPR= eig_all.sim.renderer.epr_quantum_analysis.get_frequencies().to_numpy()[:,0]

renderer_hfss.clean_active_design()
for i in range(nmodes):
    freq = freq_EPR[i]
    data['Freq_EPR'+str(i+1)+'(GHz)'] = freq
    for j in range(i):
        data['Chi'+str(i+1)+'_'+str(j+1)+'(MHz)'] = chi[i][j]

dat = pd.DataFrame(data, index = [0])
datas = pd.concat([datas,dat], ignore_index=True)


datas.to_csv('data\Two_QB_distance.csv')

Loop actual

In [ ]:
datas = pd.DataFrame()
for mode, x in enumerate(xs):
    data = {}
    nmodes = 6

    data['Pos1_x(mm)'] = 0
    data['Pos2_x(mm)'] = x
    
    design.delete_component('cpw_2')
    
    TQ2.options['pos_x'] = x-0.22
    q2.options['pos_x'] = x
    anchor = trans_p.anchor_CPW(q2, 150*u.um,0.05,2,last_offset = 0.4)
    # cpw2.options['anchors'] = anchor
    pin_inputs['start_pin']['component'] = 'Q2'
    pin_inputs['end_pin']['component'] = 'TQ2'
    CPW_options['pin_inputs'] = pin_inputs
    op = Dict(anchors = anchor, **CPW_options)
    cpw2 = RouteMixed(design, 'cpw_2', op)

    gui.rebuild()
    
    c = LOManalysis(design, "q3d")
    c.sim.setup.min_passes  = 10
    c.sim.setup.max_passes = 30
    c.sim.run(components=components_all)
    c.sim.capacitance_matrix 

    c.sim._get_results_from_renderer()
    c_mat = c.sim.capacitance_matrix
    
    c1  = c_mat['a_connector_pad_Q1']['pad_top_Q1']
    c2  = c_mat['a_connector_pad_Q1']['pad_top_Q2']
    c12  = c_mat['pad_top_Q1']['pad_top_Q2']

    data['C_Q1_Q2(fF)'] = c12
    data['C_Q1_g(fF)'] = c1
    data['C_Q2_g(fF)'] = c2
    
    eig_all = EPRanalysis(design, 'hfss')

    hfss = eig_all.sim.renderer

    eig_all.sim.renderer.options['Lj'] = '13nH'
    eig_all.sim.renderer.options['Lj1'] = '13nH'
    eig_all.sim.renderer.options['Lj2'] = '13nH'
    eig_all.sim.renderer.options['wb_threshold'] = '72um'

    eig_all.sim.setup.max_passes = 40
    eig_all.sim.setup.max_delta_f = 0.05
    eig_all.sim.setup.n_modes = nmodes
    eig_all.sim.setup.vars = Dict(Lj1= '13 nH', Cj1= '0 fF',Lj2 = '13nH', Cj2 = '0fF')
    eig_all.sim.renderer.options['x_buffer_width_mm'] = 0.5
    eig_all.sim.renderer.options['y_buffer_width_mm'] = 0.5
    eig_all.sim.run(name="all_freqs", components=components_all)#, open_terminations=[('Q1', 'a')])
    convergence = pd.read_csv('hfss_eig_f_convergence.csv')
    conv = convergence.dropna()
    
    ind = list(conv.keys())[1:]
    for i in range(nmodes):
        freq = conv[ind[(i)]].values[-1]
        data['Freq'+str(i+1)+'(GHz)'] = freq
    
    #save screenshots for future refrences
    for i in range(4):
        hfss.modeler._modeler.ShowWindow()
        hfss.set_mode(i+1,'Setup')
        hfss.plot_ansys_fields('main')
        hfss.save_screenshot()
        hfss.clear_fields(['main'])
        
        actual = r'C:\Users\slab\Desktop\Wendy-qiskit-code\s{}_m{}.png'.format(mode,i)

        os.rename(original, actual)
        dest = shutil.move(actual, path)
    #Run the EPR analysis
    del eig_all.setup.junctions['jj']
    eig_all.setup.junctions.jj1 = Dict(rect='JJ_rect_Lj_Q1_rect_jj', line='JJ_Lj_Q1_rect_jj_',
                    Lj_variable='Lj1', Cj_variable='Cj1')
    eig_all.setup.junctions.jj2 = Dict(rect='JJ_rect_Lj_Q2_rect_jj', line='JJ_Lj_Q2_rect_jj_',
                    Lj_variable='Lj2', Cj_variable='Cj2')
    eig_all.setup.sweep_variable = 'Lj1'
    eig_all.run_epr()

    #Save the EPR data
    chi = eig_all.sim.renderer.epr_quantum_analysis.get_chis().to_numpy()
    freq_EPR= eig_all.sim.renderer.epr_quantum_analysis.get_frequencies().to_numpy()[:,0]
    for i in range(nmodes):
        freq = freq_EPR[i]
        data['Freq_EPR'+str(i+1)+'(GHz)'] = freq
        for j in range(i):
            data['Chi'+str(i+1)+'_'+str(j+1)+'(MHz)'] = chi[i][j]
    
    dat = pd.DataFrame(data, index = [0])
    datas = pd.concat([datas,dat], ignore_index=True)


    datas.to_csv('data\Two_QB_distance.csv')
    

end of loop woohoo

In [ ]:
hfss.set_mode(4,'Setup')
hfss.plot_ansys_fields('main')
hfss.save_screenshot()

In [ ]:
hfss.clear_fields(['main'])

In [ ]:
data


In [ ]:
hfss.clear_fields(['Mag_E1'])

In [ ]:
cpw2.options['anchors'] = anchor
gui.rebuild()

In [ ]:
eig_all.sim.renderer.epr_quantum_analysis.get_frequencies().to_numpy()[:,0]

In [ ]:
import qutip as q
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as c
import scipy.linalg as lin

In [ ]:
r = lin.eig(lin.inv(C)@(L))


In [ ]:
np.sqrt(r[0])*1000

In [ ]:
(1/np.sqrt(13*u.nH*100*u.fF).si).to(u.GHz)/2/np.pi

In [ ]:
from Transmon_property import transmon_freq

In [ ]:
transmon_freq(110*u.fF, 13*u.nH)

# Simulations

## Capacitance Matrix Calculation

Note that when we calculate capacitance matrix, we need to ``approximate" that the end pins of the CPW is not grounded to get the capacitance coupling. Thus we need to modify the designs above to get the desired capacitance matrix. 

In [ ]:
draw_CPW(short=False)

In [ ]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [ ]:
c1.sim.setup.min_passes  = 1
c1.sim.setup.max_passes = 5
print(c1.sim.setup)

In [ ]:
c1.sim.run(components=['Q1','cpw_1','end'])#, open_terminations=[('end', 'a')])#, ('Q1', 'bus1'), ('Q1', 'bus2')])
c1.sim.capacitance_matrix 

c1.sim._get_results_from_renderer()
c_mat = c1.sim.capacitance_matrix

In [ ]:
c_mat

In the capacitance matrix above, the ground plane is named "g_wb" The big metal pad in the junction pocket is called "pad_top_Q1" The CPW is connected with the connector pad, thus its name is "a_connector_pad"

In [ ]:
c1.clear_data()

## EPR Analysis

In [ ]:
import numpy as np
from qiskit_metal.analyses.quantization import EPRanalysis

In [ ]:
hfss.clean_active_design()

In [ ]:

draw_CPW()
# design.delete_component('Q1')
eig_res = EPRanalysis(design, "hfss")

hfss = eig_res.sim.renderer
hfss.start()

hfss.activate_ansys_design('readout','eigenmode')

hfss.render_design(['cpw_1','Q1','end'],[])
hfss.save_screenshot()

setup = hfss.pinfo.setup
setup.passes = 40
setup.min_converged = 9
setup.n_modes = 2

print(f"""
Number of eigenmodes to find             = {setup.n_modes}
Number of simulation passes              = {setup.passes}
Convergence freq max delta percent diff  = {setup.delta_f}
""")

# Next 2 lines are counterinuitive, since there is no junction in this resonator.
# However, these are necessary to make pyEPR work correctly. Please do note delete
hfss.pinfo.design.set_variable('Lj', '10 nH')
hfss.pinfo.design.set_variable('Cj', '0 fF')

setup.analyze()

In [ ]:
eig_res.get_frequencies()

In [ ]:
eig_res.sim.convergence_t, eig_res.sim.convergence_f, _ = hfss.get_convergences()
eig_res.sim.plot_convergences()

In [ ]:
hfss.modeler._modeler.ShowWindow()
hfss.plot_ansys_fields('main')
hfss.save_screenshot()

In [ ]:
hfss.clear_fields(['Mag_E1'])

In [ ]:
hfss.set_mode(2, "Setup")

In [ ]:
hfss.modeler._modeler.ShowWindow()
hfss.plot_fields('main')
hfss.save_screenshot()

In [ ]:
eig_res.sim.setup

In [ ]:
eig_res.setup

In [ ]:
eig_res.setup.junctions.jj.rect = 'JJ_rect_Lj_Q1_rect_jj'
eig_res.setup.junctions.jj.line = 'JJ_Lj_Q1_rect_jj_'
eig_res.setup

In [ ]:
eig_res.run_epr()

In [ ]:
from tqdm import tqdm
import numpy as np
import time

In [ ]:
for member in tqdm(np.linspace(0,100,100)):
    time.sleep(0.1)
    print(member)

## Save Design to GDS

In [ ]:
a_gds = design.renderers.gds

In [ ]:
a_gds.options

In [ ]:
a_gds.options['path_filename'] = 'Customized_Components\Fake_Junctions.GDS'

In [ ]:

a_gds.options['short_segments_to_not_fillet'] = 'False'
a_gds.export_to_gds('One pad design.gds')

The rest of the notebook is just a scratch paper :D

In [ ]:
q3d.clean_active_design()

In [ ]:
import numpy as np
import scipy.linalg as la
import qutip as q
import astropy.units as u
import astropy.constants as con

class Single_Transmon_Circuit:
    ## Default constant
    eq = 1.602e-19*u.C 
    epsilon = con.eps0
    h = 6.636/10**34
    hbar = con.hbar
    kb = con.k_B
    Tc = 1.2
    delta = 1.764*kb*Tc
    phi0 = con.h/2/eq
    dielectric = 9.34*epsilon
    def __init__(self,cap_mat,pad_name,CPW_name,ground_plane_name = 'ground_main_plane') -> None:
        self.c_couple = -cap_mat[pad_name][CPW_name]*u.fF
        self.c_to_ground = cap_mat[pad_name][pad_name]
        self.c_CPW = cap_mat[CPW_name][CPW_name]

In [ ]:
Single_Transmon_Circuit(c_mat,'pad_top_Q1','n_spiral_spiral')